# [CptS 215 Data Analytics Systems and Algorithms](https://piazza.com/wsu/fall2017/cpts215/home)
[Washington State University](https://wsu.edu)

[Gina Sprint](http://eecs.wsu.edu/~gsprint/)
## PA2 Python for Data Analysis (100 pts)
<mark>Due:</mark>

### Learner Objectives
At the conclusion of this programming assignment, participants should be able to:
* Write objected oriented Python code
* Use the `pandas` library
* Implement k-means clustering
* Analyze a real-world dataset

### Prerequisites
Before starting this programming assignment, participants should be able to:
* Write Python code that utilizes:
    * File I/O
    * Strings
    * Lists
    * Command line arguments
* Write Markdown and code cells in Jupyter Notebook
* Type set equations using Latex
* Create plots with `matplotlib`

### Acknowledgments
Content used in this assignment is based upon information in the following sources:
* [k-Means Clustering](https://www.engage-csedu.org/find-resources/k-means-clustering) assignment by Chris Bailey-Kellogg

## Overview and Requirements
For this programming assignment, we are going to implement the [k-means clustering algorithm](https://en.wikipedia.org/wiki/K-means_clustering) in Jupyter Notebook. Cluster analysis seeks to separate objects into groups (or clusters), such that the objects within a group are more similar to each other than they are to objects in other groups. For example, we could cluster students by locations of their hometowns, stocks by a list of their closing prices over some interval of time, news articles based on the similarity of the words they contain, or types of cancer based on the profiles of genes that are active.

This problem set studies cluster analysis of data defined by an array of numerical feature values. So a student's hometown location array might just have a latitude and longitude. A stock's array would have a price for each day. For news story aggregation, each article might have the numbers of occurrences of various words. And for the example dataset that we'll actually use here, each patient will be characterized by an array of gene expression levels, indicating for each gene a kind of relative abundance for that patient.

The goal will be to take a set of such samples and find common groups (k of them!).

### Program Details
#### Background: Samples
A sample has a name and an array of values. We will will view each index in the array as a dimension, and will think of an array of n samples as an n-dimensional point. For example, [simple.csv](https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/files/simple.csv):

|Sample|Feature #1|Feature #2|Feature #3|Feature #4|Feature #5|Features #6|Feature #7|Feature #8|Feature #9|Feature #10|
|-|-|-|-|
|g0|0  |0.1|0.2|0  |0.4|0.5|0.6|0.7|0.8|0.9|
|g1|1.0|0.9|0.8|0.7|0.6|0.5|0.4|0.3|0  |0.1|
|g2|0.1|0.2|0.3|0.4|0.5|0.6|0.7|0.8|0.9|1.0|
|g3|0.4|0.4|0.4|0.4|0.4|0.4|0.4|0.4|0.4|0.4|
|g4|0.9|0.8|0.7|0.6|0.5|0.4|0.3|0.2|0.1|0.0|
|g5|0.5|0  |0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|

To visualize samples all together, we'll use a heat map with a blue-red color scale, from blue for low values up to red for high values, with white in the middle:

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/figures/simple_csv_heatmap.png" width="400" />

Upon rearranging the samples, we can see three different groups in in this toy example (separated by a black line):

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/figures/simple_csv_heatmap_clustered.png" width="400" />

#### Background: K-means Clustering
One way to cluster objects is called k-means clustering. The goal is to find k different clusters, each represented by a "prototype", defined as the centroid of cluster. The centroid is computed as follows: the jth value in the centroid is the mean (average) of the jth values of all the members of the cluster. Our goal is for every member a cluster to be closer to that cluster's prototype than to any of the other prototypes. Thus a prototype is a good representative for the items in a cluster.

We start the algorithm with k initial clusters and centroids. We then alternate between two steps:
1. For each sample, find the nearest prototype. This assigns members to each cluster.
1. Set the prototype of each cluster as the centroid of its members.

We stop when the cluster assignment doesn't change, or when we've reached a maximum number of iterations. 

Note: In June of 2009 a paper was published that showed that in the worst case the time for k-means to converge can be exponential ([Vattani, 2009](http://cseweb.ucsd.edu/~avattani/papers/kmeans-journal.pdf)), but that's rare in practice, and not a problem in this assignment.

There are many ways to initialize K-means. We will use a random approach: for each sample, assign it to a randomly selected cluster. So create a list of clusters (each of which has a list of its members). The members lists start off empty, and we just go through the samples, randomly choosing a cluster to assign them to.

Once members are assigned to a cluster, the cluster can set its prototype as the centroid. Each iteration produces a new set of clusters, by assigning the samples according to the closest prototypes. An easy way to do this is, for each sample, to determine which index into the old clusters list has the closest prototype to the sample, and then add the sample to the cluster of the same index in the new clusters list. Closest is again just by [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance):

$$d(\mathbf{p}, \mathbf{q}) = d(\mathbf{q}, \mathbf{p}) = \sqrt{(q_{1} - p_{1})^{2} + (q_{2} - p_{2})^{2} + ... + (q_{n} - p_{n})^{2}} = \sqrt{\sum_{i=1}^{n}(q_{i} - p_{i})^{2}}$$

where $\mathbf{p}$ and $\mathbf{q}$ are equal length vectors.

You are to write your own Euclidean distance function; however, you can check your work with the [Scipy `euclidean` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html).

The process iterates until it reaches convergence, that is the membership of samples to clusters does not change from iteration to the next. 

#### Gene Expression Data
In addition to the simple test dataset, you are to run your code on a real dataset on which clustering was performed to gain real insight. This data was published in the paper by van't Veer et al., ["Gene expression profiling predicts clinical outcome of breast cancer"](https://www.ncbi.nlm.nih.gov/pubmed/11823860) in Nature 2002, 415:530-536. A preprocessed and filtered version is provided as an example for the Matlab bioinformatics toolkit. Chris Bailey-Kellogg then did a little more preprocessing (normalizing the data) to form the dataset we will use: [cancer.csv](https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/files/cancer.csv). Each row in the array corresponds to a different patient, labeled as either having a recurrence of cancer or not. The columns are different genes, and the values how much more or less the gene is represented in that patient (a subset of divergent genes have been preselected). The question then is whether there is a distinctive pattern of gene expression between the two classes of patient.

Heat map of [cancer.csv](https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/files/cancer.csv):
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/figures/cancer_csv_heatmap.png" width="400" />

Heat map of [cancer.csv]() after 2-means clustering:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/figures/cancer_csv_heatmap_clustered.png" width="400" />

The two clusters are separated by a black line in the figure. We see that one group of subjects has more blue in the left bunch of genes (columns) while the other has more red, and vice-versa in the right bunch of genes. These genes appear to be some "fingerprint" distinguishing the two classes of patients. (They were selected for that distinction.)

#### Data Storage
Read in the gene expression dataset into a `pandas` dataframe using [`read_csv()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). The first column in the input file is the index. The columns are unlabeled.

Note: the simple made-up case for initial testing is [simple.csv](https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/files/simple.csv); the real dataset is [cancer.csv](https://raw.githubusercontent.com/gsprint23/cpts215/master/progassignments/files/cancer.csv).

#### Data Visualization
Plot the dataset using a heat map. From `matplotlib`, you can use either [`imshow()`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow) or [`pcolor()`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.pcolor) to produce heat maps similar to the ones shown above. You can check out the [`matplotlib color references`](http://matplotlib.org/examples/color/colormaps_reference.html) example to see what color maps are available to choose from!

#### Classes to Define
You are free to decide which classes and methods/functions to define to solve this problem. Break the problem into objects and tasks and build your solution accordingly. You will be graded on your approach. Ask the instructor or your TA for help with this!

## Bonus (5 pts)
Do multiple random restarts of k-means and return the result that is best in "some sense". Justify your decision.

## Submitting Assignments
1. Use the Blackboard tool https://learn.wsu.edu to submit your assignment. You will submit your code to the corresponding programming assignment under the "Content" tab. You must upload your solutions as `<your last name>_pa2.zip` by the due date and time.
2. Your .zip file should contain your .ipynb file and the .csv files you used to text your program.

## Grading Guidelines
This assignment is worth 100 points + 5 points bonus. Your assignment will be evaluated based on a successful compilation and adherence to the program requirements. We will grade according to the following criteria:
* 10 pts for correctly storing the sample data in a `DataFrame`
* 10 pts for correctly initializing random clusters (ensure an empty cluster does not exist)
* 10 pts for correctly computing the Euclidean distance
* 10 pts for correctly computing cluster prototype (centroid)
* 10 pts for correctly determining the closest prototype
* 15 pts for correctly re-clustering using iteration
* 5 pts for correct re-clustering convergence condition
* 10 pts for plotting the sample data using heat maps
* 10 pts for appropriate OOP design and decomposition of functions/methods
* 5 pts for including a discussion of clustering patterns, similarities and differences, strengths and weaknesses
* 5 pts for adherence to proper programming style and comments established for the class